In [ ]:
from tensorflow.keras import layers , activations , models , preprocessing , utils
import tensorflow as tf
from  sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import os
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('gutenberg')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


True

In [ ]:
df1 = pd.read_csv('dữ liệu chatbot question-answer short style.csv',index_col = 0)
df1.head(5)

,user_a,user_b
0,Thích mẫu người nào,"Dễ thương, tóc dài, da trắng"
1,Có crush ai không,Có 1 bạn cùng lớp
2,Tại sao lại thích bạn dó,Vì đáp ứng những yêu cầu của tao
3,Có hay nói chuyện không,Hay nhắn tin messenger
4,Bạn kia có bắt chuyện trước không,Có đôi khi


In [ ]:
df2 = pd.read_csv('chatbot data.csv',index_col = 0)
df2.head(5)

,user_a,user_b
0,bạn tên gì?,Mình tên Trường <eos> còn bạn tên gì?
1,Tui tên Nam <eos> Bạn học ở đâu?,tui học ở TDTU nè <eos> còn bạn học ở trường n...
2,tui học ở TDTU luôn <eos> Bạn học ngành gì,mình học Công Nghệ Thông Tin
3,Chuyên ngành gì vậy bạn ?,mình học bên Kỹ Thuật Phần Mềm á <eos> còn bạn...
4,mình học Quản Trị Kinh Doanh <eos> Bạn là ngườ...,Mình là người Đồng tháp <eos> bạn biết Đồng Th...


# PreProcessing

In [ ]:
import re
pair = []
for i in range(df2.shape[0]):
    ques = re.sub(r'[^\w\d\s]','',str(df2.iloc[i]['user_a']).strip().lower())
    ans = re.sub(r'[^\w\d\s]','',str(df2.iloc[i]['user_b']).strip().lower())
    user_a = ques.split(' eos ')
    user_b = ans.split(' eos ')

    if len(user_b) > 1:
        #lấy câu hỏi user_a vs câu trl đầu tiên trong user_b
        pair.append( [user_a[-1].split(' ') , user_b[0].split(' ')] )
        #lấy câu sau của user_b làm câu hỏi và câu tiếp theo của user_a làm cau trl
        pair.append( [user_b[-1].split(' ') , df2.iloc[i+1]['user_a'].strip().split(' <eos> ')[0].lower().split(' ')] )
    elif user_b[0] != 'nan':
        pair.append( [user_a[-1].split(' ') , user_b[0].split(' ')] )

    #một dố câu bị NaN thì bỏ qua
print(pair[:10])




[[['bạn', 'tên', 'gì'], ['mình', 'tên', 'trường']], [['còn', 'bạn', 'tên', 'gì'], ['tui', 'tên', 'nam']], [['bạn', 'học', 'ở', 'đâu'], ['tui', 'học', 'ở', 'tdtu', 'nè']], [['còn', 'bạn', 'học', 'ở', 'trường', 'nào', ''], ['tui', 'học', 'ở', 'tdtu', 'luôn']], [['bạn', 'học', 'ngành', 'gì'], ['mình', 'học', 'công', 'nghệ', 'thông', 'tin']], [['chuyên', 'ngành', 'gì', 'vậy', 'bạn', ''], ['mình', 'học', 'bên', 'kỹ', 'thuật', 'phần', 'mềm', 'á']], [['còn', 'bạn', 'học', 'ngành', 'gì', ''], ['mình', 'học', 'quản', 'trị', 'kinh', 'doanh']], [['bạn', 'là', 'người', 'ở', 'đâu', 'vậy', ''], ['mình', 'là', 'người', 'đồng', 'tháp']], [['bạn', 'biết', 'đồng', 'tháp', 'không', ''], ['mình', 'biết,', 'đồng', 'tháp', 'dưới', 'miền', 'tây', 'đúng', 'không', '?']], [['mình', 'biết', 'đồng', 'tháp', 'dưới', 'miền', 'tây', 'đúng', 'không', ''], ['đúng', 'rồi', 'đó']]]


In [ ]:
df1 = df1.dropna()

In [ ]:
for i in range(df1.shape[0]-3000):
    #thêm data của data1 vào list
    ques = re.sub(r'[^\w\d\s]','',df1.iloc[i]['user_a'].strip().lower())
    ans = re.sub(r'[^\w\d\s]','',df1.iloc[i]['user_b'].strip().lower())
    pair.append([ques.split(' '), ans.split(' ')])

In [ ]:
print(pair[:3])

[[['bạn', 'tên', 'gì'], ['mình', 'tên', 'trường']], [['còn', 'bạn', 'tên', 'gì'], ['tui', 'tên', 'nam']], [['bạn', 'học', 'ở', 'đâu'], ['tui', 'học', 'ở', 'tdtu', 'nè']]]


# BLEU SCORE

In [ ]:
import nltk.translate.bleu_score as bleu
smooth = bleu.SmoothingFunction().method4

def BleuScore(ques):
    scores = [bleu.sentence_bleu([pair[i][0]], ques, weights=(0.5,0.5,0), smoothing_function= smooth) for i in range(len(pair))]
    s_max = max(scores)
    ans = pair[scores.index(s_max)][1]
    return ' '.join(ans)

In [ ]:
chat = True
while chat:
    ques = str(input('Question: '))
    if ques == 'q':
        chat = False
    else:
        ans = BleuScore(ques.split())
        print('Answer: {}'.format(ans))

Question: bạn tên là gì
Answer: tui tên là đậu
Question: q


# Word2Vec


In [ ]:
from gensim.models import Word2Vec


In [ ]:
question_tokens = [i[0] for i in pair]
print(len(pair))
print(len(question_tokens))
print(question_tokens[:3])

4541
4541
[['bạn', 'tên', 'gì'], ['còn', 'bạn', 'tên', 'gì'], ['bạn', 'học', 'ở', 'đâu']]


CBOW


In [ ]:
#CBOW
crow = Word2Vec(question_tokens,size=150, window=5, min_count=2, workers=4, sg=0)

In [ ]:
#hàm chuyển word thành vecter
def word2vecCrow(seq):
    vec = np.zeros(150,dtype='float')
    for word in seq:
        if word not in crow.wv:
            continue
        vec += crow.wv[word]
    return vec/len(seq)

In [ ]:
w2c = [word2vecCrow(i) for i in question_tokens]

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

vec_input = word2vecCrow("bạn học ở đâu và ngành gì".split())
q = euclidean_distances([vec_input],w2c)
predict = np.argmin(q,axis=1)
print(' '.join(pair[predict[0]][1]))

hải sản


Skip Grams

In [ ]:
#Skip-grams
skip = Word2Vec(question_tokens,size=150, window=5, min_count=2, workers=4, sg=1)

In [ ]:
def word2vecSkip(seq):
    vec = np.ones(150,dtype='float')
    for word in seq:
        deno = np.ones(150,dtype='float')
        if word not in skip.wv:
            continue
        vec += crow.wv[word]
    return vec

In [ ]:
w2c_Skip = [word2vecSkip(i) for i in question_tokens]

In [ ]:
vec_input = word2vecSkip("bạn học trường nào".split())
q = euclidean_distances([vec_input],w2c_Skip)
predict = np.argmin(q,axis=1)
print(' '.join(pair[predict[0]][1]))

tớ học đại học tôn đức thắng


# Pad_Sequences

In [ ]:
ans_tokens = [['<start>']+ i[1]+['<end>'] for i in pair]
print(ans_tokens[:3])

[['<start>', 'mình', 'tên', 'trường', '<end>'], ['<start>', 'tui', 'tên', 'nam', '<end>'], ['<start>', 'tui', 'học', 'ở', 'tdtu', 'nè', '<end>']]


In [ ]:
question_tokens = [['<start>']+ i+['<end>'] for i in question_tokens]
print(question_tokens[:3])

[['<start>', 'bạn', 'tên', 'gì', '<end>'], ['<start>', 'còn', 'bạn', 'tên', 'gì', '<end>'], ['<start>', 'bạn', 'học', 'ở', 'đâu', '<end>']]


In [ ]:
from tensorflow.keras import preprocessing, utils
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(question_tokens + ans_tokens)
VOCAB_SIZE = len(tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

VOCAB SIZE : 3104


In [ ]:
#encoder dữ liệu đầu vào
tokenized_questions = tokenizer.texts_to_sequences(question_tokens)
maxlen_questions = max( [len(x) for x in tokenized_questions ])
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions, maxlen = maxlen_questions, padding = 'post')
encoder_input_data = np.array(padded_questions)
print(encoder_input_data.shape)


(4541, 73)


In [ ]:
# decoder dữ liệu đầu vào
tokenized_answers = tokenizer.texts_to_sequences(ans_tokens)
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answer = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answer )
print( decoder_input_data.shape)

(4541, 36)


In [ ]:
tokenized_answers = tokenizer.texts_to_sequences(ans_tokens)
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]

padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
onehot_answers = utils.to_categorical( padded_answers)
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape )

(4541, 36, 3104)


# LSTM

In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding

# Define an input sequence and process it.
encoder_inputs = Input(shape=(maxlen_questions,))
encoder_embedding = Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(200 , return_state=True )(encoder_embedding)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]


# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(maxlen_answers,))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_embedding = Embedding(VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = LSTM(200, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding,
                                     initial_state=encoder_states)
decoder_dense = Dense(VOCAB_SIZE, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)


# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 73)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 36)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 73, 200)      620800      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 36, 200)      620800      ['input_2[0][0]']                
                                                                                              

In [ ]:
model.fit([encoder_input_data, decoder_input_data], decoder_output_data,
          batch_size=50,
          epochs=150)

Epoch 1/150
91/91 [==============================] - 11s 29ms/step - loss: 0.9881
Epoch 2/150
91/91 [==============================] - 3s 29ms/step - loss: 0.8836
Epoch 3/150
91/91 [==============================] - 3s 28ms/step - loss: 0.8575
Epoch 4/150
91/91 [==============================] - 3s 28ms/step - loss: 0.8346
Epoch 5/150
91/91 [==============================] - 3s 28ms/step - loss: 0.8117
Epoch 6/150
91/91 [==============================] - 3s 29ms/step - loss: 0.7918
Epoch 7/150
91/91 [==============================] - 3s 29ms/step - loss: 0.7729
Epoch 8/150
91/91 [==============================] - 3s 29ms/step - loss: 0.7548
Epoch 9/150
91/91 [==============================] - 3s 29ms/step - loss: 0.7363
Epoch 10/150
91/91 [==============================] - 3s 30ms/step - loss: 0.7186
Epoch 11/150
91/91 [==============================] - 3s 31ms/step - loss: 0.7008
Epoch 12/150
91/91 [==============================] - 3s 29ms/step - loss: 0.6833
Epoch 13/150
91/91 [====

In [ ]:
from keras.models import load_model

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(200,))
decoder_state_input_c = Input(shape=(200,))

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [ ]:
 encoder_model = load_model('encoder.h5')
 decoder_model = load_model('decoder.h5')

In [ ]:
def str_tokens(sentence):
  words = ['<start>'] + sentence.lower().split() + ['<end>']
  tokens_list = list()
  for word in words:
    tokens_list.append(tokenizer.word_index[word])
  return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post' )

In [ ]:
states_values = encoder_model.predict(str_tokens( input( 'Enter question : ' ) ) )
empty_target_seq = np.zeros( ( 1 , 1 ) )
empty_target_seq[0, 0] = tokenizer.word_index['<start>']
condition = True
decoded_translation = ''
while condition :
    dec_outputs , h , c = decoder_model.predict([ empty_target_seq ] + states_values )
    sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
    sampled_word = None
    for word , index in tokenizer.word_index.items() :
        if sampled_word_index == index :
            decoded_translation += ' {}'.format( word )
            sampled_word = word

    if sampled_word == '<end>' or len(decoded_translation.split()) > maxlen_answers:
        condition = False
        if sampled_word == '<end>':
          decoded_translation = decoded_translation[:-5]

    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[ 0 , 0 ] = sampled_word_index
    states_values = [ h , c ]

print( decoded_translation )

Enter question : bạn quê ở đâu
 mình quê ở tiền giang 


# Transformer


In [ ]:
class TokenEmbedding(layers.Layer):
    def __init__(self, num_vocab=1000, maxlen=100, num_hid=64):
        super().__init__()
        self.emb = tf.keras.layers.Embedding(num_vocab, num_hid)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=num_hid)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        x = self.emb(x)
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        return x + positions


In [ ]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, num_heads, feed_forward_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.enc = keras.Sequential(
            [
                layers.Dense(feed_forward_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        enc_output = self.enc(out1)
        enc_output = self.dropout2(enc_output, training=training)
        return self.layernorm2(out1 + enc_output)

In [ ]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, num_heads, feed_forward_dim, dropout_rate=0.1):
        super().__init__()
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)
        self.self_att = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.enc_att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.self_dropout = layers.Dropout(0.5)
        self.enc_dropout = layers.Dropout(0.1)
        self.enc = keras.Sequential(
            [
                layers.Dense(feed_forward_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )

    def causal_attention_mask(self, batch_size, n_dest, n_src, dtype):
        """Masks the upper half of the dot product matrix in self attention.

        This prevents flow of information from future tokens to current token.
        1's in the lower triangle, counting from the lower right corner.
        """
        i = tf.range(n_dest)[:, None]
        j = tf.range(n_src)
        m = i >= j - n_src + n_dest
        mask = tf.cast(m, dtype)
        mask = tf.reshape(mask, [1, n_dest, n_src])
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
        )
        return tf.tile(mask, mult)

    def call(self, enc_out, target):
        input_shape = tf.shape(target)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = self.causal_attention_mask(batch_size, seq_len, seq_len, tf.bool)
        target_att = self.self_att(target, target, attention_mask=causal_mask)
        target_norm = self.layernorm1(target + self.self_dropout(target_att))
        enc_out = self.enc_att(target_norm, enc_out)
        enc_out_norm = self.layernorm2(self.enc_dropout(enc_out) + target_norm)
        enc_out = self.enc(enc_out_norm)
        enc_out_norm = self.layernorm3(enc_out_norm + self.enc_dropout(enc_out))
        return enc_out_norm

In [ ]:
from tensorflow import keras
class Transformer(keras.Model):
    def __init__(
        self,
        num_hid=50,
        num_head=2,
        num_feed_forward=128,
        source_maxlen=100,
        target_maxlen=100,
        num_layers_enc=4,
        num_layers_dec=1,
        num_classes=10,
    ):
        super().__init__()
        self.loss_metric = keras.metrics.Mean(name="loss")
        self.num_layers_enc = num_layers_enc
        self.num_layers_dec = num_layers_dec
        self.num_classes = num_classes
        self.source_maxlen=source_maxlen
        self.target_maxlen=target_maxlen
        self.enc_input = TokenEmbedding(
            num_vocab=VOCAB_SIZE,num_hid=num_hid, maxlen=source_maxlen)
        self.dec_input = TokenEmbedding(
            num_vocab=VOCAB_SIZE, maxlen=target_maxlen, num_hid=num_hid
        )

        self.encoder = keras.Sequential(
            [self.enc_input]
            + [
                TransformerEncoder(num_hid, num_head, num_feed_forward)
                for _ in range(num_layers_enc)
            ]
        )

        for i in range(num_layers_dec):
            setattr(
                self,
                f"dec_layer_{i}",
                TransformerDecoder(num_hid, num_head, num_feed_forward),
            )

        self.classifier = layers.Dense(num_classes)

    def decode(self, enc_out, target):
        y = self.dec_input(target)
        for i in range(self.num_layers_dec):
            y = getattr(self, f"dec_layer_{i}")(enc_out, y)
        return y

    def call(self, inputs):
        source = inputs[0]
        target = inputs[1]
        x = self.encoder(source)
        y = self.decode(x, target)
        return self.classifier(y)

    @property
    def metrics(self):
        return [self.loss_metric]

    def train_step(self, batch):
        """Processes one batch inside model.fit()."""
        source = batch["source"]
        target = batch["target"]
        dec_input = target[:, :-1]
        dec_target = target[:, 1:]
        with tf.GradientTape() as tape:
            preds = self([source, dec_input])
            one_hot = tf.one_hot(dec_target, depth=self.num_classes)
            mask = tf.math.logical_not(tf.math.equal(dec_target, 0))
            loss = self.compiled_loss(one_hot, preds, sample_weight=mask)
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        self.loss_metric.update_state(loss)
        return {"loss": self.loss_metric.result()}

    def test_step(self, batch):
        source = batch["source"]
        target = batch["target"]
        dec_input = target[:, :-1]
        dec_target = target[:, 1:]
        preds = self([source, dec_input])
        one_hot = tf.one_hot(dec_target, depth=self.num_classes)
        mask = tf.math.logical_not(tf.math.equal(dec_target, 0))
        loss = self.compiled_loss(one_hot, preds, sample_weight=mask)
        self.loss_metric.update_state(loss)
        return {"loss": self.loss_metric.result()}

    def generate(self, source, target_start_token_idx):
        """Performs inference over one batch of inputs using greedy decoding."""
        bs = tf.shape(source)[0]
        enc = self.encoder(source)
        dec_input = tf.ones((bs, 1), dtype=tf.int32) * target_start_token_idx
        dec_logits = []
        for i in range(self.target_maxlen - 1):
            dec_out = self.decode(enc, dec_input)
            logits = self.classifier(dec_out)
            logits = tf.argmax(logits, axis=-1, output_type=tf.int32)
            last_logit = tf.expand_dims(logits[:, -1], axis=-1)
            dec_logits.append(last_logit)
            dec_input = tf.concat([dec_input, last_logit], axis=-1)
        return dec_input

In [ ]:
class DisplayOutputs(keras.callbacks.Callback):
    def __init__(
        self, batch, idx_to_token, target_start_token_idx=27, target_end_token_idx=28
    ):
        """Displays a batch of outputs after every epoch

        Args:
            batch: A test batch containing the keys "source" and "target"
            idx_to_token: A List containing the vocabulary tokens corresponding to their indices
            target_start_token_idx: A start token index in the target vocabulary
            target_end_token_idx: An end token index in the target vocabulary
        """
        self.batch = batch
        self.target_start_token_idx = target_start_token_idx
        self.target_end_token_idx = target_end_token_idx
        self.idx_to_char = idx_to_token

    def on_epoch_end(self, epoch, logs=None):
        if epoch % 5 != 0:
            return
        source = self.batch["source"]
        target = self.batch["target"]
        bs = tf.shape(source)[0]
        preds = self.model.generate(source, self.target_start_token_idx)
        preds = preds.numpy()
        print(preds)
        for i in range(bs):
            target_text = "".join([self.idx_to_char[_] for _ in target[i, :]])
            prediction = ""
            for idx in preds[i, :]:
                prediction += self.idx_to_char[idx]
                if idx == self.target_end_token_idx:
                    break
            print(f"target:     {target_text.replace('-','')}")
            print(f"prediction: {prediction}\n")

In [ ]:

class CustomSchedule(keras.optimizers.schedules.LearningRateSchedule):
    def __init__(
        self,
        init_lr=0.00001,
        lr_after_warmup=0.001,
        final_lr=0.00001,
        warmup_epochs=15,
        decay_epochs=85,
        steps_per_epoch=203,
    ):
        super().__init__()
        self.init_lr = init_lr
        self.lr_after_warmup = lr_after_warmup
        self.final_lr = final_lr
        self.warmup_epochs = warmup_epochs
        self.decay_epochs = decay_epochs
        self.steps_per_epoch = steps_per_epoch

    def calculate_lr(self, epoch):
        """ linear warm up - linear decay """
        warmup_lr = (
            self.init_lr
            + ((self.lr_after_warmup - self.init_lr) / (self.warmup_epochs - 1)) * epoch
        )
        decay_lr = tf.math.maximum(
            self.final_lr,
            self.lr_after_warmup
            - (epoch - self.warmup_epochs)
            * (self.lr_after_warmup - self.final_lr)
            / (self.decay_epochs),
        )
        return tf.math.minimum(warmup_lr, decay_lr)

    def __call__(self, step):
        epoch = step // self.steps_per_epoch
        return self.calculate_lr(epoch)

In [ ]:

def create_text_ds(data):
    texts = [_ for _ in data]
    text_ds = tf.data.Dataset.from_tensor_slices(texts)
    return text_ds


def create_tf_dataset(ques,ans,bs=4):
    ques_ds = create_text_ds(ques)
    ans_ds = create_text_ds(ans)
    ds = tf.data.Dataset.zip((ques_ds, ans_ds))
    ds = ds.map(lambda x, y: {"source": x, "target": y})
    ds = ds.batch(bs)
    return ds


In [ ]:
split = int(len(encoder_input_data) * 0.99)
ques_train = encoder_input_data[:split]
ques_test = encoder_input_data[split:]

ans_train = decoder_input_data[:split]
ans_test = decoder_input_data[split:]
ds = create_tf_dataset(ques_train,ans_train, bs=64)
val_ds = create_tf_dataset(ques_test,ans_test, bs=4)

In [ ]:
model = Transformer(
    num_hid=200,
    num_head=2,
    num_feed_forward=400,
    source_maxlen=maxlen_questions,
    target_maxlen=maxlen_answers,
    num_layers_enc=4,
    num_layers_dec=1,
    num_classes=VOCAB_SIZE,
)
loss_fn = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True, label_smoothing=0.1,
)

learning_rate = CustomSchedule(
    init_lr=0.01,
    lr_after_warmup=0.1,
    final_lr=0.01,
    warmup_epochs=15,
    decay_epochs=85,
    steps_per_epoch=len(question_tokens),
)
optimizer = keras.optimizers.Adam(learning_rate)
model.compile(optimizer=optimizer, loss=loss_fn)


In [ ]:
history = model.fit(ds, validation_data=val_ds,epochs=150)


Epoch 1/150
71/71 [==============================] - 11s 75ms/step - loss: 1.2826 - val_loss: 0.6982
Epoch 2/150
71/71 [==============================] - 5s 64ms/step - loss: 1.2385 - val_loss: 0.6596
Epoch 3/150
71/71 [==============================] - 5s 64ms/step - loss: 1.1986 - val_loss: 0.6310
Epoch 4/150
71/71 [==============================] - 5s 64ms/step - loss: 1.1680 - val_loss: 0.6112
Epoch 5/150
71/71 [==============================] - 5s 64ms/step - loss: 1.1446 - val_loss: 0.5968
Epoch 6/150
71/71 [==============================] - 5s 65ms/step - loss: 1.1259 - val_loss: 0.5858
Epoch 7/150
71/71 [==============================] - 5s 65ms/step - loss: 1.1103 - val_loss: 0.5771
Epoch 8/150
71/71 [==============================] - 5s 65ms/step - loss: 1.0969 - val_loss: 0.5699
Epoch 9/150
71/71 [==============================] - 5s 65ms/step - loss: 1.0853 - val_loss: 0.5639
Epoch 10/150
71/71 [==============================] - 5s 64ms/step - loss: 1.0749 - val_loss: 0.558